<ol>
    <li>Попробуйте обучить CNN на датасете CIFAR-10.Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
   
</ol>

### Import libraries

In [41]:
import tensorflow as tf

# Display the version
print(tf.__version__)

# other imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

2.12.0


### Load Data

In [3]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

170498071/170498071 [==============================] - 2s 0us/step


### Basic model (Sequential API)

In [30]:
model_basic = tf.keras.models.Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])
model_basic.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_basic.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                 

### Model with BN (Sequential API)

In [31]:
model_BN = tf.keras.models.Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model_BN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_BN.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_19 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_3 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 64)       

### Preatrained model (Functional API)

In [32]:
def feature_extractor(inputs):

    mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(32,32,3),
    include_top=False,
    weights='imagenet'
)

    feature_extractor = mobilenet_model(inputs)

    return feature_extractor

def dense_layers(features):

    x = Flatten()(features)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(10, activation='softmax')(x)

    return x

def final_model(inputs):

    feature_cnn = feature_extractor(inputs)

    last_dense_layer = dense_layers(feature_cnn)

    model =tf.keras.Model(inputs=inputs, outputs=last_dense_layer)

    return model

def define_and_compile_model():

    inputs = tf.keras.Input(shape=(32,32,3))

    model = final_model(inputs)

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [33]:
model_pretrained = define_and_compile_model()

model_pretrained.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1, 1, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_7 (Flatten)         (None, 1280)              0         
                                                                 
 dropout_14 (Dropout)        (None, 1280)              0         
                                                                 
 dense_14 (Dense)            (None, 1024)              1311744   
                                                                 
 dropout_15 (Dropout)        (None, 1024)              0         
                                                           

### Training and evaluating without augmentations

In [34]:
bacis_res = model_basic.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)
scores_bs = model_basic.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_bs[0])
print('Test accuracy:', scores_bs[1])

Epoch 1/20
1563/1563 [==============================] - 16s 7ms/step - loss: 1.5065 - accuracy: 0.4462 - val_loss: 1.1640 - val_accuracy: 0.5704
Epoch 2/20
1563/1563 [==============================] - 10s 6ms/step - loss: 1.0292 - accuracy: 0.6323 - val_loss: 0.9134 - val_accuracy: 0.6770
Epoch 3/20
1563/1563 [==============================] - 10s 6ms/step - loss: 0.8273 - accuracy: 0.7089 - val_loss: 0.7623 - val_accuracy: 0.7335
Epoch 4/20
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7012 - accuracy: 0.7520 - val_loss: 0.7063 - val_accuracy: 0.7572
Epoch 5/20
1563/1563 [==============================] - 10s 6ms/step - loss: 0.6229 - accuracy: 0.7814 - val_loss: 0.7431 - val_accuracy: 0.7430
Epoch 6/20
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5541 - accuracy: 0.8075 - val_loss: 0.6940 - val_accuracy: 0.7687
Epoch 7/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.4903 - accuracy: 0.8275 - val_loss: 0.7593 - val_ac

In [35]:
BN_res = model_BN.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)
scores_bn = model_BN.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_bn[0])
print('Test accuracy:', scores_bn[1])

Epoch 1/20
1563/1563 [==============================] - 17s 8ms/step - loss: 1.3853 - accuracy: 0.5185 - val_loss: 1.3404 - val_accuracy: 0.5453
Epoch 2/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.9107 - accuracy: 0.6841 - val_loss: 1.0004 - val_accuracy: 0.6707
Epoch 3/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7388 - accuracy: 0.7440 - val_loss: 0.8489 - val_accuracy: 0.7238
Epoch 4/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.6306 - accuracy: 0.7826 - val_loss: 0.6976 - val_accuracy: 0.7641
Epoch 5/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5339 - accuracy: 0.8148 - val_loss: 0.6554 - val_accuracy: 0.7852
Epoch 6/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.4563 - accuracy: 0.8418 - val_loss: 0.6684 - val_accuracy: 0.7798
Epoch 7/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.3909 - accuracy: 0.8621 - val_loss: 0.7051 - val_ac

In [36]:
pretr_res = model_pretrained.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)
scores_pt = model_pretrained.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_pt[0])
print('Test accuracy:', scores_pt[1])

Epoch 1/20
1563/1563 [==============================] - 114s 51ms/step - loss: 1.3829 - accuracy: 0.5390 - val_loss: 1.4700 - val_accuracy: 0.5230
Epoch 2/20
1563/1563 [==============================] - 56s 36ms/step - loss: 0.9997 - accuracy: 0.6745 - val_loss: 1.0482 - val_accuracy: 0.6731
Epoch 3/20
1563/1563 [==============================] - 50s 32ms/step - loss: 0.9327 - accuracy: 0.6970 - val_loss: 2.6416 - val_accuracy: 0.4209
Epoch 4/20
1563/1563 [==============================] - 51s 33ms/step - loss: 1.0133 - accuracy: 0.6619 - val_loss: 2.1225 - val_accuracy: 0.5360
Epoch 5/20
1563/1563 [==============================] - 46s 30ms/step - loss: 0.9080 - accuracy: 0.7008 - val_loss: 1.9576 - val_accuracy: 0.5351
Epoch 6/20
1563/1563 [==============================] - 46s 30ms/step - loss: 0.8891 - accuracy: 0.7099 - val_loss: 2.2332 - val_accuracy: 0.3907
Epoch 7/20
1563/1563 [==============================] - 46s 29ms/step - loss: 0.7879 - accuracy: 0.7409 - val_loss: 1.8409 

### Training and evaluating with augmentations

In [37]:
datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(x_train)

In [38]:
bacis_res = model_basic.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=20, validation_data=(x_test, y_test))
scores_bs_aug = model_basic.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_bs_aug[0])
print('Test accuracy:', scores_bs_aug[1])

Epoch 1/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.8139 - accuracy: 0.7267 - val_loss: 0.7769 - val_accuracy: 0.7486
Epoch 2/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7488 - accuracy: 0.7448 - val_loss: 0.7712 - val_accuracy: 0.7522
Epoch 3/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7350 - accuracy: 0.7459 - val_loss: 0.6910 - val_accuracy: 0.7669
Epoch 4/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7250 - accuracy: 0.7525 - val_loss: 0.7500 - val_accuracy: 0.7581
Epoch 5/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7107 - accuracy: 0.7565 - val_loss: 0.7054 - val_accuracy: 0.7721
Epoch 6/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7051 - accuracy: 0.7588 - val_loss: 0.7289 - val_accuracy: 0.7622
Epoch 7/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7006 - accuracy: 0.7600 - val_loss: 0.7200 -

In [39]:
BN_res = model_BN.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=20, validation_data=(x_test, y_test))
scores_bn_aug = model_BN.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_bn_aug[0])
print('Test accuracy:', scores_bn_aug[1])

Epoch 1/20
1563/1563 [==============================] - 37s 23ms/step - loss: 0.6250 - accuracy: 0.7930 - val_loss: 0.7228 - val_accuracy: 0.7794
Epoch 2/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.5121 - accuracy: 0.8280 - val_loss: 0.6687 - val_accuracy: 0.7976
Epoch 3/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.4674 - accuracy: 0.8410 - val_loss: 0.6662 - val_accuracy: 0.7937
Epoch 4/20
1563/1563 [==============================] - 37s 24ms/step - loss: 0.4476 - accuracy: 0.8479 - val_loss: 0.5448 - val_accuracy: 0.8217
Epoch 5/20
1563/1563 [==============================] - 37s 24ms/step - loss: 0.4269 - accuracy: 0.8536 - val_loss: 0.5378 - val_accuracy: 0.8285
Epoch 6/20
1563/1563 [==============================] - 37s 24ms/step - loss: 0.4115 - accuracy: 0.8595 - val_loss: 0.6041 - val_accuracy: 0.8163
Epoch 7/20
1563/1563 [==============================] - 36s 23ms/step - loss: 0.3945 - accuracy: 0.8641 - val_loss: 0.5842 -

In [40]:
pretr_res = model_pretrained.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=20, validation_data=(x_test, y_test))
scores_pt_aug = model_pretrained.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_pt_aug[0])
print('Test accuracy:', scores_pt_aug[1])

Epoch 1/20
1563/1563 [==============================] - 84s 51ms/step - loss: 1.3280 - accuracy: 0.5434 - val_loss: 1.3317 - val_accuracy: 0.5349
Epoch 2/20
1563/1563 [==============================] - 71s 45ms/step - loss: 1.3311 - accuracy: 0.5391 - val_loss: 1.2260 - val_accuracy: 0.5652
Epoch 3/20
1563/1563 [==============================] - 75s 48ms/step - loss: 1.2135 - accuracy: 0.5846 - val_loss: 1.5791 - val_accuracy: 0.4939
Epoch 4/20
1563/1563 [==============================] - 69s 44ms/step - loss: 1.3032 - accuracy: 0.5511 - val_loss: 1.2863 - val_accuracy: 0.5491
Epoch 5/20
1563/1563 [==============================] - 70s 45ms/step - loss: 1.3624 - accuracy: 0.5270 - val_loss: 1.2535 - val_accuracy: 0.5640
Epoch 6/20
1563/1563 [==============================] - 69s 44ms/step - loss: 1.1858 - accuracy: 0.5950 - val_loss: 1.1544 - val_accuracy: 0.5944
Epoch 7/20
1563/1563 [==============================] - 69s 44ms/step - loss: 1.1151 - accuracy: 0.6199 - val_loss: 1.3649 -

In [43]:
res = pd.DataFrame({'Model': ['Basic CNN','CNN with BatchNormalization','Pretrained CNN','Basic CNN_Aug','CNN with BatchNormalization_Aug','Pretrained CNN_Aug'],
              'Test accuracy': [scores_bs[1],scores_bn[1],scores_pt[1],scores_bs_aug[1],scores_bn_aug[1],scores_pt_aug[1]],
              'Test_loss': [scores_bs[0],scores_bn[0],scores_pt[0],scores_bs_aug[0],scores_bn_aug[0],scores_pt_aug[0]]})
res.sort_values(['Test accuracy'],ascending=False)

,Model,Test accuracy,Test_loss
4,CNN with BatchNormalization_Aug,0.8366,0.541678
1,CNN with BatchNormalization,0.8099,0.895071
3,Basic CNN_Aug,0.7913,0.652551
0,Basic CNN,0.7762,0.918997
2,Pretrained CNN,0.4853,1.510943
5,Pretrained CNN_Aug,0.4037,1.687052


___________________


В итоге, положительным образом сказались добавление оптимизационного слоя (BatchNormalization) и аугментаций (хотя их неправильный подбор может ухужшить результат). Transfer Learning на примере MobileNet_V2 должно было значительно увеличить результат, т.к. имеются уже оптимальные веса, но что-то пошло не так...